In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import numpy as np
from collections import defaultdict
import os
import shutil
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from tensorflow.keras.models import load_model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os

# Diretórios
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/test'

img_size = (224, 224)
batch_size = 32

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1, 0.1),
])

# Carregamento dos dados
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=img_size, batch_size=batch_size, label_mode='categorical')
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=img_size, batch_size=batch_size, label_mode='categorical')
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=img_size, batch_size=batch_size, label_mode='categorical')

class_names = test_ds.class_names

# Pré-processamento
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(data_augmentation(x)), y)).cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(tf.data.AUTOTUNE)

# Modelo base ResNet50
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

# Cabeça do modelo
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compilação inicial
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("best_resnet_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)

# Treinamento inicial
history_initial = model.fit(train_ds, validation_data=val_ds, epochs=10,
                            callbacks=[early_stop])

# Fine-tuning: liberar todas as camadas
for layer in base_model.layers:
    layer.trainable = True

# Recompilar com taxa de aprendizado menor
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinamento com fine-tuning + melhorias
history_finetune = model.fit(train_ds, validation_data=val_ds, initial_epoch=10, epochs=25,
                             callbacks=[early_stop, lr_scheduler, checkpoint])

# Avaliação final
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\n✅ Final Test Accuracy (ResNet50): {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
# Após o treinamento
model.save("resnet50_finetuned_model.h5")


In [ ]:
loss = history.history['loss'] + history_finetune.history['loss']
val_loss = history.history['val_loss'] + history_finetune.history['val_loss'] 

plt.figure(figsize=(8, 5))
plt.plot(loss, label='Treino')
plt.plot(val_loss, label='Validação')
plt.title('Evolução da perda (loss)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# Avaliação no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\n✅ Test Accuracy (ResNet50): {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
